In [ ]:
#!pip install webdriver-manager

In [1]:
import re
import pandas as pd
from pandas import DataFrame 

import openpyxl
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver 
from selenium.common.exceptions import NoSuchElementException 
from selenium.webdriver.common.keys import Keys

# 시간
from tqdm import tqdm 
import time

#이미지 다운
import urllib.request

#오류
import traceback

In [2]:
#driver = webdriver.Chrome('C:/Users/Administrator/jpnt/Final Project/chromedriver')

## Webdirver option 설정
options = webdriver.ChromeOptions()
options.add_argument('headless') #크롬 띄우기창 없애기
options.add_argument('window-size=1920x1080') # 크롬드라이버 창크기 
options.add_argument("disable-gpu") #그래픽 성능 낮춰서 크롤링 성능 쪼금 높이기 
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36") # 네트워크 설정 
options.add_argument("lang=ko_KR") # 사이트 주언어 
driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\Song Kyung Jun\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_16532/4267179865.py:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)


In [4]:
#검색할 데이터 명
df = pd.read_excel("path/data.xlsx")

In [5]:
data_list = []
# 포문
for idx, row in tqdm(df.iterrows()): 
    cd_idx = row['cd_idx']
    search_name = row['search_list'] 
    try:
        #영상 업로드 시간을 한정하기 위해 자동으로 click하게 만든 코드
        youtubeUrl = (f"https://www.youtube.com/results?search_query={search_name}") 
        driver.get(youtubeUrl)
        time.sleep(1.5)
        driver.find_element_by_xpath('//*[@id="container"]/ytd-toggle-button-renderer/a').click()
        time.sleep(0.6)
        driver.find_element_by_css_selector("#collapse-content ytd-search-filter-group-renderer:nth-child(1) ytd-search-filter-renderer:nth-of-type(3) a").click()
        time.sleep(0.6)
        
        no_of_pagedowns = 300 #유튜브에서 한 검색어당 500개 이상은 잘 안나옴
        elem = driver.find_element_by_tag_name("body") 
        while no_of_pagedowns: 
            elem.send_keys(Keys.PAGE_DOWN) 
            time.sleep(0.5) 
            no_of_pagedowns -= 1
        boxs = driver.find_elements_by_id("dismissible")


        for box in boxs: 
            title = box.find_element_by_css_selector('#video-title') 
            link = title.get_attribute("href") 
            title_name = box.find_element_by_css_selector('#video-title').text 
            img_ = box.find_element_by_css_selector('#img') 
            img = img_.get_attribute('src')
            while img is None: #None값을 없애기 위해 강제적으로 while문 사용
                img = img_.get_attribute('src')

            name = box.find_element_by_css_selector('#channel-info').text  
            regdate_ment = box.find_element_by_css_selector('#metadata-line > span:nth-child(2)').text 
            viewcount = box.find_element_by_css_selector('#metadata-line > span:nth-child(1)').text 
            
            
            data = {
                'search_list': search_name, 
                    "link": link,
                    "title_name": title_name,
                    "image": img, "name": name, 
                    "regdate_ment": regdate_ment, "viewcount": viewcount}
            data_list.append(data)


    except: #렉 걸려서 멈췄을 때를 대비하기 위해
        print(idx) #몇번째에서 렉걸렸고 못 받아온건지 알기 위한 코드
        continue

result_df = pd.DataFrame(data_list,columns=[
    'search_list', 'link',
    'title_name', 'image','name', 
    'regdate_ment', 'viewcount']) 

1it [03:09, 189.83s/it]

0


3it [11:04, 223.36s/it]

2



4it [14:17, 211.38s/it]

...

9it [33:58, 238.62s/it]

8



10it [37:04, 222.49s/it]

9


12it [45:50, 229.17s/it]


In [6]:
result_df.to_excel(f'path/211105_result.xlsx', index=False)

In [3]:
#읽어오기, 중복값 제거
result_df = pd.read_excel('path/211105_result.xlsx')
result_df_drop = result_df.drop_duplicates()
#index reset하고 열 드랍
rir = result_df_drop.reset_index()
rir = rir.drop(['index'], axis=1)
rir
rir.to_excel(f'path/rir_fof.xlsx', index=False)

In [4]:
#None 조회에는 is를 쓴다.
pd.value_counts(result_df['search_list'])

순대 먹방       503
분모자 먹방      494
삼겹살 먹방      492
김치찌개 먹방     488
카레 먹방       482
볶음밥 먹방      476
푸드파이터 먹방    196
먹방 ASMR     173
곱창 먹방       120
중국당면 먹방      39
라면 먹방         3
디저트 먹방        3
Name: search_list, dtype: int64

In [5]:
len(rir)

3457

In [3]:
rir = pd.read_excel('path/rir_fof.xlsx')

In [10]:

for idx, row in rir[3000:].iterrows():
    s_names = row['link']
    urls = (f"{s_names}") 
    try: 
        driver.get(urls)
        time.sleep(2)
        subscribers = driver.find_element_by_xpath('//*[@id="owner-sub-count"]').text
        alls = driver.find_element_by_class_name('style-scope ytd-video-primary-info-renderer')
        likes = alls.find_element_by_css_selector('#text').text
        sub_data = {'subscribers':subscribers, 'likes':likes}
        subs_list.append(sub_data)
    except:
        print(idx)
        continue
subs_df = pd.DataFrame(subs_list, columns=['subscribers','likes'])

3286


In [ ]:
subs_df_all = pd.concat([subs_df_2, subs_df], ignore_index=True)

In [8]:
subs_df_2 = subs_df

In [14]:
subs_df_all.to_excel('path/211105.xlsx', index=False)

In [23]:
subs_df = pd.read_excel('path/211105.xlsx')

In [24]:
rir = pd.read_excel('path/rir_fof.xlsx')

In [25]:
rir = rir.dropna(axis=0)
rir=rir.reset_index()
rir = rir.drop(['index'], axis=1)

In [ ]:
all_df = pd.concat([rir, subs_df], axis=1)

In [27]:
all_df.to_excel('path/211106.xlsx', index=False)

In [2]:
all_df = pd.read_excel('path/211106.xlsx', keep_default_na =False)

In [29]:
all_df['subscribers'][67]

'구독자 5.97만명'

In [50]:
pd.value_counts(all_df['subscribers'] == '')

False    2652
True      804
Name: subscribers, dtype: int64

In [ ]:
all_df = all_df[all_df.subscribers != '']

In [8]:
all_df = all_df.reset_index()
all_df = all_df.drop(['index'], axis=1)

In [10]:
# viewcount 전처리
for i in range(len(all_df)):
    if "없음" in all_df["viewcount"][i]:
        all_df["viewcount"][i] = '0'
    elif "만회" in all_df["viewcount"][i]:
        num = re.findall("\d+", all_df["viewcount"][i])[0]
        all_df["viewcount"][i] = float(num)*10000

    elif "천회" in all_df["viewcount"][i]:
        num = re.findall("\d+", all_df["viewcount"][i])[0]
        all_df["viewcount"][i] = float(num)*1000

    elif "회" in all_df["viewcount"][i]:
        num = re.findall("\d+", all_df["viewcount"][i])[0]
        all_df["viewcount"][i] = float(num)

In [11]:
# regdate_ment 전처리 업로드 날짜만 int로
for i in range(len(all_df)):
    if "분" in all_df["regdate_ment"][i]: # 분 추가함
        all_df["regdate_ment"][i] = 1
    elif "시간" in all_df["regdate_ment"][i]: 
        all_df["regdate_ment"][i] = 1

    elif "일" in all_df["regdate_ment"][i]:
        num = re.findall("\d+", all_df["regdate_ment"][i])[0] #re가 리스트 형식으로 문자열을 불러옴 따라서 문자열만 출력
        all_df["regdate_ment"][i] = num

    elif "주" in all_df["regdate_ment"][i]:
        num = re.findall("\d+", all_df["regdate_ment"][i])[0]
        all_df["regdate_ment"][i] = int(num)*7 # 문자열을 숫자로 바꾼 후 계산후 문자열로 변환

    elif "개월" in all_df["regdate_ment"][i]:
        num = re.findall("\d+", all_df["regdate_ment"][i])[0]
        all_df["regdate_ment"][i] = int(num)*30

In [12]:
# likes 전처리
for i in range(len(all_df)):
    if '.' in all_df['likes'][i]:
        if "만" in all_df["likes"][i]:
            t = all_df['likes'][i][:-1]
            all_df["likes"][i] = float(t)*10000

        elif "천" in all_df["likes"][i]:
            t = all_df['likes'][i][:-1]
            all_df["likes"][i] = float(t)*1000
    elif '' == all_df["likes"][i]:
        all_df["likes"][i] = 0.
    elif '좋아요' in all_df['likes'][i]:
        all_df['likes'][i] = 0
    elif "만" in all_df["likes"][i]:
        num = re.findall("\d+", all_df["likes"][i])[0]
        all_df["likes"][i] = float(num)*10000

    elif "천" in all_df["likes"][i]:
        num = re.findall("\d+", all_df["likes"][i])[0]
        all_df["likes"][i] = float(num)*1000
    else:
        int(all_df['likes'][i])

In [13]:
# subscribers 전처리
for i in range(len(all_df)):
    if '.' in all_df['subscribers'][i]:
        if ("만명" in all_df["subscribers"][i]) or ('만' in all_df["subscribers"][i]):
            t = all_df['subscribers'][i][3:-2]
            all_df["subscribers"][i] = float(t)*10000

        elif ("천명" in all_df["subscribers"][i]) or ('천' in all_df["subscribers"][i]):
            t = all_df['subscribers'][i][3:-2]
            all_df["subscribers"][i] = float(t)*1000

#    elif '' == all_df["subscribers"][i]:
#        all_df["subscribers"][i] = 0.

    elif ("만명" in all_df["subscribers"][i]) or ('만' in all_df["subscribers"][i]):
        num = re.findall("\d+", all_df["subscribers"][i])[0]
        all_df["subscribers"][i] = float(num)*10000

    elif ("천명" in all_df["subscribers"][i]) or ('천' in all_df["subscribers"][i]):
        num = re.findall("\d+", all_df["subscribers"][i])[0]
        all_df["subscribers"][i] = float(num)*1000

    else:
        num = re.findall("\d+", all_df["subscribers"][i])[0]
        all_df["subscribers"][i] = num

In [14]:
all_df.to_excel('path/211106_preall.xlsx', index=False)

In [28]:
pre_all = pd.read_excel('path/211106_preall.xlsx')

In [ ]:
pre_all_filtered = pre_all[~pre_all['title_name'].str.contains('중국 먹방', case=False)]
pre_all_filtered = pre_all_filtered[~pre_all['title_name'].str.contains('tiktok', case=False)]
pre_all_filtered = pre_all_filtered[~pre_all['title_name'].str.contains('tik tok', case=False)]

In [ ]:
pre_all_filtered = pre_all_filtered[pre_all_filtered['subscribers'] <= 2000000]

In [ ]:
pre_all_filtered = pre_all_filtered.drop(['index'], axis=1)
pre_all_filtered['index'] = pre_all_filtered.index

In [35]:
pre_all_filtered['index'] = pre_all_filtered['index'] + 7503

In [37]:
pre_all_filtered.to_excel('path/211106_preall_fil.xlsx', index=False)

In [38]:
#이미지 저장 코드
#pass / continue : continue 는 바로 다음 iteerows로
#pass 는 다음 명령어로
for idx, row in pre_all_filtered.iterrows():
    try:
        img = row['image']
        urllib.request.urlretrieve(img, f"{row['search_list'],row['index']}.jpg")
    except:
        print(idx)

In [30]:
pre_all_filtered=pd.read_excel('path/211106_preall_fil.xlsx')
pre_all_filtered = pre_all_filtered.dropna(axis=0)
pre_all_filtered = pre_all_filtered.reset_index()

In [63]:
pre_all_filtered=pd.read_excel('path/pre_all_filtered.xlsx')